## 1. setup

In [14]:
from datetime import datetime as dt
import csv, os, glob
from pprint import pprint

import matplotlib.pyplot as plt
import plotly.express as px

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras as k
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt


## 2. pre-process data

In [54]:
# Split
split_index = int(len(light_data)*0.9)
train_data, test_data = norm_data[:split_index], norm_data[split_index:]
train_label, test_label = label_data[:split_index], label_data[split_index:]

test_data.reset_index(drop=True, inplace=True)
test_label.reset_index(drop=True, inplace=True)

NameError: name 'light_data' is not defined

In [55]:
print(train_data.shape, train_label.shape)
print(test_data.shape, test_label.shape)

print(train_data[100:105])
print(train_label[100:105])
print(test_data[100:105])
print(test_label[100:105])

NameError: name 'train_data' is not defined

## 4. make & train model

In [9]:
input = k.Input(shape=(80,80,1), batch_size=1)

x = k.layers.Conv2D(filters=32, kernel_size=(3,3), strides=(2,2), padding='same', activation='relu')(input)
x = k.layers.DepthwiseConv2D(kernel_size=32, strides=(1,1), depth_multiplier=1, padding='same', activation='relu')(x)
x = k.layers.Conv2D(filters=24, kernel_size=(1,1), strides=(1,1), padding='same')(x)

x = k.layers.Conv2D(filters=144, kernel_size=(1,1), strides=(1,1), padding='same', activation='relu')(x)
x = k.layers.DepthwiseConv2D(kernel_size=144, strides=(1,1), depth_multiplier=1, padding='same', activation='relu')(x)
x0 = k.layers.Conv2D(filters=32, kernel_size=(1,1), strides=(1,1), padding='same')(x)

x = k.layers.Conv2D(filters=192, kernel_size=(1,1), strides=(1,1), padding='same', activation='relu')(x0)
x = k.layers.DepthwiseConv2D(kernel_size=192, strides=(1,1), depth_multiplier=1, padding='same', activation='relu')(x)
x = k.layers.Conv2D(filters=32, kernel_size=(1,1), strides=(1,1), padding='same')(x)
add = k.layers.Add()([x, x0])

x = k.layers.Conv2D(filters=192, kernel_size=(1,1), strides=(1,1), padding='same', activation='relu')(add)
x = k.layers.DepthwiseConv2D(kernel_size=192, strides=(1,1), depth_multiplier=1, padding='same', activation='relu')(x)
x = k.layers.Conv2D(filters=32, kernel_size=(1,1), strides=(1,1), padding='same')(x)
add = k.layers.Add()([x, add])

x0 = k.layers.Conv2D(filters=192, kernel_size=(1,1), strides=(1,1), padding='same', activation='relu')(add)
x = k.layers.DepthwiseConv2D(kernel_size=192, strides=(2,2), depth_multiplier=1, padding='same', activation='relu')(x0)
x1 = k.layers.Conv2D(filters=64, kernel_size=(1,1), strides=(1,1), padding='same')(x)

x = k.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), padding='same', activation='relu')(x1)
x = k.layers.DepthwiseConv2D(kernel_size=384, strides=(1,1), depth_multiplier=1, padding='same', activation='relu')(x)
x = k.layers.Conv2D(filters=64, kernel_size=(1,1), strides=(1,1), padding='same')(x)
add = k.layers.Add()([x, x1])

x = k.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), padding='same', activation='relu')(add)
x = k.layers.DepthwiseConv2D(kernel_size=384, strides=(1,1), depth_multiplier=1, padding='same', activation='relu')(x)
x = k.layers.Conv2D(filters=64, kernel_size=(1,1), strides=(1,1), padding='same')(x)
add = k.layers.Add()([x, add])

x = k.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), padding='same', activation='relu')(add)
x = k.layers.DepthwiseConv2D(kernel_size=384, strides=(1,1), depth_multiplier=1, padding='same', activation='relu')(x)
x = k.layers.Conv2D(filters=96, kernel_size=(1,1), strides=(1,1), padding='same')(x)
x1 = k.layers.Conv2D(filters=576, kernel_size=(1,1), strides=(1,1), padding='same', activation='relu')(x)

x = k.layers.Conv2D(filters=128, kernel_size=(1,1), strides=(1,1), padding='same', activation='relu')(x1)
x2 = k.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(2,2), padding='same', activation='relu')(x)
x = k.layers.Conv2D(filters=128, kernel_size=(1,1), strides=(1,1), padding='same', activation='relu')(x2)
x3 = k.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(2,2), padding='same', activation='relu')(x)

l1 = k.layers.Conv2D(filters=12, kernel_size=(1,1), strides=(1,1), padding='same')(x0)
l11 = k.layers.Reshape((4800,1,4))(l1)
l2 = k.layers.Conv2D(filters=32, kernel_size=(1,1), strides=(1,1), padding='same')(x1)
l22 = k.layers.Reshape((3200,1,4))(l2)
l3 = k.layers.Conv2D(filters=32, kernel_size=(1,1), strides=(1,1), padding='same')(x2)
l33 = k.layers.Reshape((800,1,4))(l3)
l4 = k.layers.Conv2D(filters=32, kernel_size=(1,1), strides=(1,1), padding='same')(x3)
l44 = k.layers.Reshape((200,1,4))(l4)

y = k.layers.Concatenate(axis=1)([l11,l22,l33,l44])
a = k.layers.Reshape((9000,4))(y)

l5 = k.layers.Conv2D(filters=6, kernel_size=(1,1), strides=(1,1), padding='same')(x0)
l55 = k.layers.Reshape((4800,2))(l5)
l6 = k.layers.Conv2D(filters=16, kernel_size=(1,1), strides=(1,1), padding='same')(x1)
l66 = k.layers.Reshape((3200,2))(l6)
l7 = k.layers.Conv2D(filters=16, kernel_size=(1,1), strides=(1,1), padding='same')(x2)
l77 = k.layers.Reshape((800,2))(l7)
l8 = k.layers.Conv2D(filters=16, kernel_size=(1,1), strides=(1,1), padding='same')(x3)
l88 = k.layers.Reshape((200,2))(l8)

y0 = k.layers.Concatenate(axis=1)([l55,l66,l77,l88])
r = k.activations.sigmoid(y0)


aa = tf.



output = k.layers.Concatenate()([a, r])

model = k.Model(input, output)

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

2022-02-09 17:30:15.659476: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 17:30:15.659634: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/z/GWT/gap_sdk/install/workstation/lib
2022-02-09 17:30:15.659695: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/z/GWT/gap_sdk/install/workstation/lib
2022-02-09 17:30:15.659849: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object fi

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(1, 80, 80, 1)]     0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (1, 40, 40, 32)      320         input_1[0][0]                    
__________________________________________________________________________________________________
depthwise_conv2d (DepthwiseConv (1, 40, 40, 32)      32800       conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (1, 40, 40, 24)      792         depthwise_conv2d[0][0]           
______________________________________________________________________________________________

In [ ]:

# load tensorboard
%load_ext tensorboard

# fit
log_path = "logs/" + dt.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = k.callbacks.TensorBoard(log_dir=log_path, histogram_freq=1)
es = EarlyStopping(monitor="val_loss", patience=10, mode="auto", verbose=2)

history = model.fit(x_train, y_train, validation_split=0.1, batch_size=6, epochs=1000, verbose=1, callbacks=[es]) # callbacks=[es, tensorboard_callback])
print(history)

# plot
pd.DataFrame(history.history).plot(figsize=(16,10), grid=1, xlabel="epoch", ylabel="accuracy")
plt.show()

In [117]:
## save model
model_name = "pplcntr_" + dt.now().strftime("%Y%m%d-%H%M%S")
model.save(model_name, save_format='h5')

## 5. evaluate & predict model

In [ ]:
# evauate model
# model_name = "/home/z/PycharmProjects/light_onoff_detector/model/light_detector_20210811-135021.h5"
model = k.models.load_model(model_name)
loss, acc = model.evaluate(x_test, y_test, verbose=1)
# print(f'test_loss: {loss} test_accuracy: {acc}')

In [ ]:
# # launch tensorboard @ localhost:6006
# %tensorboard --logdir logs/ --host localhost --port 6006

In [12]:
"""Exports an SSD detection model to use with tf-lite. See export_tflite_ssd_graph.py for usage."""
import os
import tempfile
import numpy as np
import tensorflow.compat.v1 as tf
from tensorflow.core.framework import attr_value_pb2
from tensorflow.core.framework import types_pb2
from tensorflow.core.protobuf import saver_pb2
from object_detection import exporter
from object_detection.builders import graph_rewriter_builder
from object_detection.builders import model_builder
from object_detection.builders import post_processing_builder
from object_detection.core import box_list
from object_detection.utils import tf_version

_DEFAULT_NUM_CHANNELS = 3
_DEFAULT_NUM_COORD_BOX = 4

if tf_version.is_tf1():
    from tensorflow.tools.graph_transforms import TransformGraph  # pylint: disable=g-import-not-at-top


def get_const_center_size_encoded_anchors(anchors):
    """Exports center-size encoded anchors as a constant tensor.
    Args:
      anchors: a float32 tensor of shape [num_anchors, 4] containing the anchor
        boxes
    Returns:
      encoded_anchors: a float32 constant tensor of shape [num_anchors, 4]
      containing the anchor boxes.
    """
    anchor_boxlist = box_list.BoxList(anchors)
    y, x, h, w = anchor_boxlist.get_center_coordinates_and_sizes()
    num_anchors = y.get_shape().as_list()

    with tf.Session() as sess:
        y_out, x_out, h_out, w_out = sess.run([y, x, h, w])
    encoded_anchors = tf.constant(np.transpose(np.stack((y_out, x_out, h_out, w_out))), dtype=tf.float32, shape=[num_anchors[0], _DEFAULT_NUM_COORD_BOX], name='anchors')
    return encoded_anchors


def append_postprocessing_op(frozen_graph_def,
                             max_detections,
                             max_classes_per_detection,
                             nms_score_threshold,
                             nms_iou_threshold,
                             num_classes,
                             scale_values,
                             detections_per_class=100,
                             use_regular_nms=False,
                             additional_output_tensors=()):
  """Appends postprocessing custom op.
  Args:
    frozen_graph_def: Frozen GraphDef for SSD model after freezing the
      checkpoint
    max_detections: Maximum number of detections (boxes) to show
    max_classes_per_detection: Number of classes to display per detection
    nms_score_threshold: Score threshold used in Non-maximal suppression in
      post-processing
    nms_iou_threshold: Intersection-over-union threshold used in Non-maximal
      suppression in post-processing
    num_classes: number of classes in SSD detector
    scale_values: scale values is a dict with following key-value pairs
      {y_scale: 10, x_scale: 10, h_scale: 5, w_scale: 5} that are used in decode
        centersize boxes
    detections_per_class: In regular NonMaxSuppression, number of anchors used
      for NonMaxSuppression per class
    use_regular_nms: Flag to set postprocessing op to use Regular NMS instead of
      Fast NMS.
    additional_output_tensors: Array of additional tensor names to output.
      Tensors are appended after postprocessing output.
  Returns:
    transformed_graph_def: Frozen GraphDef with postprocessing custom op
    appended
    TFLite_Detection_PostProcess custom op node has four outputs:
    detection_boxes: a float32 tensor of shape [1, num_boxes, 4] with box
    locations
    detection_classes: a float32 tensor of shape [1, num_boxes]
    with class indices
    detection_scores: a float32 tensor of shape [1, num_boxes]
    with class scores
    num_boxes: a float32 tensor of size 1 containing the number of detected
    boxes
  """
  new_output = frozen_graph_def.node.add()
  new_output.op = 'TFLite_Detection_PostProcess'
  new_output.name = 'TFLite_Detection_PostProcess'
  new_output.attr['_output_quantized'].CopyFrom(attr_value_pb2.AttrValue(b=True))
  new_output.attr['_output_types'].list.type.extend([types_pb2.DT_FLOAT, types_pb2.DT_FLOAT, types_pb2.DT_FLOAT, types_pb2.DT_FLOAT])
  new_output.attr['_support_output_type_float_in_quantized_op'].CopyFrom(attr_value_pb2.AttrValue(b=True))
  new_output.attr['max_detections'].CopyFrom(attr_value_pb2.AttrValue(i=max_detections))
  new_output.attr['max_classes_per_detection'].CopyFrom(attr_value_pb2.AttrValue(i=max_classes_per_detection))
  new_output.attr['nms_score_threshold'].CopyFrom(attr_value_pb2.AttrValue(f=nms_score_threshold.pop()))
  new_output.attr['nms_iou_threshold'].CopyFrom(attr_value_pb2.AttrValue(f=nms_iou_threshold.pop()))
  new_output.attr['num_classes'].CopyFrom(attr_value_pb2.AttrValue(i=num_classes))
  new_output.attr['y_scale'].CopyFrom(attr_value_pb2.AttrValue(f=scale_values['y_scale'].pop()))
  new_output.attr['x_scale'].CopyFrom(attr_value_pb2.AttrValue(f=scale_values['x_scale'].pop()))
  new_output.attr['h_scale'].CopyFrom(attr_value_pb2.AttrValue(f=scale_values['h_scale'].pop()))
  new_output.attr['w_scale'].CopyFrom(attr_value_pb2.AttrValue(f=scale_values['w_scale'].pop()))
  new_output.attr['detections_per_class'].CopyFrom(attr_value_pb2.AttrValue(i=detections_per_class))
  new_output.attr['use_regular_nms'].CopyFrom(attr_value_pb2.AttrValue(b=use_regular_nms))

  new_output.input.extend(['raw_outputs/box_encodings', 'raw_outputs/class_predictions', 'anchors'])
  # Transform the graph to append new postprocessing op
  input_names = []
  output_names = ['TFLite_Detection_PostProcess']+list(additional_output_tensors)
  transforms = ['strip_unused_nodes']
  transformed_graph_def = TransformGraph(frozen_graph_def, input_names, output_names, transforms)
  return transformed_graph_def


def export_tflite_graph(pipeline_config,
                        trained_checkpoint_prefix,
                        output_dir,
                        add_postprocessing_op,
                        max_detections,
                        max_classes_per_detection,
                        detections_per_class=100,
                        use_regular_nms=False,
                        binary_graph_name='tflite_graph.pb',
                        txt_graph_name='tflite_graph.pbtxt',
                        additional_output_tensors=()):
  """Exports a tflite compatible graph and anchors for ssd detection model.
  Anchors are written to a tensor and tflite compatible graph
  is written to output_dir/tflite_graph.pb.
  Args:
    pipeline_config: a pipeline.proto object containing the configuration for
      SSD model to export.
    trained_checkpoint_prefix: a file prefix for the checkpoint containing the
      trained parameters of the SSD model.
    output_dir: A directory to write the tflite graph and anchor file to.
    add_postprocessing_op: If add_postprocessing_op is true: frozen graph adds a
      TFLite_Detection_PostProcess custom op
    max_detections: Maximum number of detections (boxes) to show
    max_classes_per_detection: Number of classes to display per detection
    detections_per_class: In regular NonMaxSuppression, number of anchors used
      for NonMaxSuppression per class
    use_regular_nms: Flag to set postprocessing op to use Regular NMS instead of
      Fast NMS.
    binary_graph_name: Name of the exported graph file in binary format.
    txt_graph_name: Name of the exported graph file in text format.
    additional_output_tensors: Array of additional tensor names to output.
      Additional tensors are appended to the end of output tensor list.
  Raises:
    ValueError: if the pipeline config contains models other than ssd or uses an
      fixed_shape_resizer and provides a shape as well.
  """
  tf.gfile.MakeDirs(output_dir)
  if pipeline_config.model.WhichOneof('model') != 'ssd':
      raise ValueError('Only ssd models are supported in tflite. Found {} in config'.format(pipeline_config.model.WhichOneof('model')))

  num_classes = pipeline_config.model.ssd.num_classes
  nms_score_threshold = {pipeline_config.model.ssd.post_processing.batch_non_max_suppression.score_threshold}
  nms_iou_threshold = {pipeline_config.model.ssd.post_processing.batch_non_max_suppression.iou_threshold}
  scale_values = {}
  scale_values['y_scale'] = {pipeline_config.model.ssd.box_coder.faster_rcnn_box_coder.y_scale}
  scale_values['x_scale'] = {pipeline_config.model.ssd.box_coder.faster_rcnn_box_coder.x_scale}
  scale_values['h_scale'] = {pipeline_config.model.ssd.box_coder.faster_rcnn_box_coder.height_scale}
  scale_values['w_scale'] = {pipeline_config.model.ssd.box_coder.faster_rcnn_box_coder.width_scale}

  image_resizer_config = pipeline_config.model.ssd.image_resizer
  image_resizer = image_resizer_config.WhichOneof('image_resizer_oneof')
  num_channels = _DEFAULT_NUM_CHANNELS
  if image_resizer == 'fixed_shape_resizer':
      height = image_resizer_config.fixed_shape_resizer.height
      width = image_resizer_config.fixed_shape_resizer.width
      if image_resizer_config.fixed_shape_resizer.convert_to_grayscale:
          num_channels = 1
      shape = [1, height, width, num_channels]
  else:
      raise ValueError('Only fixed_shape_resizer is supported with tflite. Found {}'.format(image_resizer_config.WhichOneof('image_resizer_oneof')))

  image = tf.placeholder(tf.float32, shape=shape, name='normalized_input_image_tensor')

  detection_model = model_builder.build(pipeline_config.model, is_training=False)
  predicted_tensors = detection_model.predict(image, true_image_shapes=None)
  # The score conversion occurs before the post-processing custom op
  _, score_conversion_fn = post_processing_builder.build(pipeline_config.model.ssd.post_processing)
  class_predictions = score_conversion_fn(predicted_tensors['class_predictions_with_background'])

  with tf.name_scope('raw_outputs'):
      # 'raw_outputs/box_encodings': a float32 tensor of shape [1, num_anchors, 4]
      #  containing the encoded box predictions. Note that these are raw
      #  predictions and no Non-Max suppression is applied on them and
      #  no decode center size boxes is applied to them.
      tf.identity(predicted_tensors['box_encodings'], name='box_encodings')
      # 'raw_outputs/class_predictions': a float32 tensor of shape
      #  [1, num_anchors, num_classes] containing the class scores for each anchor
      #  after applying score conversion.
      tf.identity(class_predictions, name='class_predictions')
  # 'anchors': a float32 tensor of shape
  #   [4, num_anchors] containing the anchors as a constant node.
  tf.identity(get_const_center_size_encoded_anchors(predicted_tensors['anchors']), name='anchors')

  # Add global step to the graph, so we know the training step number when we
  # evaluate the model.
  tf.train.get_or_create_global_step()

  # graph rewriter
  is_quantized = pipeline_config.HasField('graph_rewriter')
  if is_quantized:
      graph_rewriter_config = pipeline_config.graph_rewriter
      graph_rewriter_fn = graph_rewriter_builder.build(graph_rewriter_config, is_training=False)
      graph_rewriter_fn()

  if pipeline_config.model.ssd.feature_extractor.HasField('fpn'):
      exporter.rewrite_nn_resize_op(is_quantized)

  # freeze the graph
  saver_kwargs = {}
  if pipeline_config.eval_config.use_moving_averages:
      saver_kwargs['write_version'] = saver_pb2.SaverDef.V1
      moving_average_checkpoint = tempfile.NamedTemporaryFile()
      exporter.replace_variable_values_with_moving_averages(tf.get_default_graph(), trained_checkpoint_prefix, moving_average_checkpoint.name)
      checkpoint_to_use = moving_average_checkpoint.name
  else:
      checkpoint_to_use = trained_checkpoint_prefix

  saver = tf.train.Saver(**saver_kwargs)
  input_saver_def = saver.as_saver_def()
  frozen_graph_def = exporter.freeze_graph_with_def_protos(
      input_graph_def=tf.get_default_graph().as_graph_def(),
      input_saver_def=input_saver_def,
      input_checkpoint=checkpoint_to_use,
      output_node_names=','.join(['raw_outputs/box_encodings', 'raw_outputs/class_predictions', 'anchors'] + list(additional_output_tensors)),
      restore_op_name='save/restore_all',
      filename_tensor_name='save/Const:0',
      clear_devices=True,
      output_graph='',
      initializer_nodes='')

  # Add new operation to do post processing in a custom op (TF Lite only)
  if add_postprocessing_op:
      transformed_graph_def = append_postprocessing_op(frozen_graph_def, max_detections, max_classes_per_detection, nms_score_threshold, nms_iou_threshold, num_classes, scale_values, detections_per_class, use_regular_nms, additional_output_tensors=additional_output_tensors)
  else:
      # Return frozen without adding post-processing custom op
      transformed_graph_def = frozen_graph_def

  binary_graph = os.path.join(output_dir, binary_graph_name)
  with tf.gfile.GFile(binary_graph, 'wb') as f:
      f.write(transformed_graph_def.SerializeToString())
  txt_graph = os.path.join(output_dir, txt_graph_name)
  with tf.gfile.GFile(txt_graph, 'w') as f:
      f.write(str(transformed_graph_def))

ModuleNotFoundError: No module named 'keras.api'